# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Sara Taylor
#### Repo: https://github.com/DrSTaylor/44620-Mod-4

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

## Preparation

In [3]:
# Create and activate a Python virtual environment. 
# Before starting the project, try all these imports FIRST
# Address any errors you get running this code cell 
# by installing the necessary packages into your active Python environment.
# Try to resolve issues using your materials and the web.
# If that doesn't work, ask for help in the discussion forums.
# You can't complete the exercises until you import these - start early! 
# We also import json and pickle (included in the Python Standard Library).

import json
import pickle

import requests
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

print('All prereqs installed.')
!pip list

All prereqs installed.
Package                       Version
----------------------------- ---------------
alabaster                     0.7.12
anaconda-client               1.11.2
anaconda-navigator            2.4.0
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.11.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.12.2
BespON                        0.6.0
binaryornot                   0.4.4
black               

## Question 1

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [4]:
import requests
import json

#result = json.loads(requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul').text)

# Setting up Lyrics Genius
import lyricsgenius
genius = lyricsgenius.Genius("SAvIdZPPcLWC3FKXLRNXYwfO6vtMXezvZJ26tOYQPEM6h0ZnBB5AQxid-V1Y8HWF")

# Setting the artist variable to the corrrect artist
artist = genius.search_artist("Rush", max_songs=5, sort="title")

# Find the song
song = artist.song('Tom Sawyer')

# Saving the Lyrics
lyrics = song.lyrics

# Creating the dictionary with song information
song_dict = {
    'Artist': 'Rush',
    'Song': 'Tom Sawyer',
    'Lyrics': lyrics
}

with open('tomsawyer_lyrics.json', 'w') as new_file:
    json.dump(song_dict, new_file)


Searching for songs by Rush...

Song 1: "2112"
Song 2: "2112 [All the World’s a Stage]"
Song 3: "2112 [Clockwork Angels Tour]"
Song 4: "2112 [Different Stages Live]"
Song 5: "2112: Discovery"

Reached user-specified song limit (5).
Done. Found 5 songs.
Searching for "Tom Sawyer" by Rush...
Done.


## Question 2

2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [5]:
with open ('tomsawyer_lyrics.json', 'r') as tomsawyer_lyrics:
    lyric = json.load(tomsawyer_lyrics)

lyrics = lyric["Lyrics"]

print("Lyrics:")
print(lyrics)

Lyrics:
68 ContributorsTom Sawyer Lyrics[Intro]
A modern-day warrior
Mean, mean stride
Today's Tom Sawyer
Mean, mean pride

[Verse 1]
Though his mind is not for rent
Don't put him down as arrogant
His reserve a quiet defense
Riding out the day's events
The river

[Chorus]
What you say about his company
Is what you say about society
Catch the mist
Catch the myth
Catch the mystery
Catch the drift

[Bridge]
The world is, the world is
Love and life are deep
Maybe as his skies are wide
Today's Tom Sawyer, he gets high on you
And the space he invades, he gets by on you
You might also like[Instrumental]

[Verse 2]
No, his mind is not for rent
To any god or government
Always hopeful, yet discontent
He knows changes aren't permanent
But change is

[Chorus]
And what you say about his company
Is what you say about society
Catch the witness
Catch the wit
Catch the spirit
Catch the spit

[Refrain]
The world is, the world is
Love and life are deep
Maybe as his eyes are wide

[Outro]
Exit the warrior

In [6]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

doc = nlp(lyrics)

# Getting Sentiment
print('Polarity Score: ', doc._.blob.polarity)

Polarity Score:  1.0582010582001663e-05


I am not surprised that this is viewed as positive from lyrics alone. Without the music to accompany it is hard to tell the meaning.

## Question 3

3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [9]:
import lyricsgenius
import json
import os

def pull_lyrics_from_api(artist, song, filename):
    access_token = 'SAvIdZPPcLWC3FKXLRNXYwfO6vtMXezvZJ26tOYQPEM6h0ZnBB5AQxid-V1Y8HWF'
    genius = lyricsgenius.Genius(access_token)
    artist = genius.search_artist(artist, max_songs=1, get_full_info=False)
    song = artist.song(song)
    with open(filename+'_lyrics.json', 'w') as filename:
        json.dump(song.lyrics, filename)
        
pull_lyrics_from_api("Tool", "Schism", "schism")
pull_lyrics_from_api("Fleetwood Mac", "Landslide", "landslide")
pull_lyrics_from_api("The Grateful Dead", "Box of Rain", "rain")
pull_lyrics_from_api("The Beatles", "Here Comes the Sun", "sun")

Searching for songs by Tool...

Song 1: "Sober"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Schism" by Tool...
Done.
Searching for songs by Fleetwood Mac...

Song 1: "Dreams"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Landslide" by Fleetwood Mac...
Done.
Searching for songs by The Grateful Dead...

Song 1: "Terrapin Station"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Box of Rain" by The Grateful Dead...
Done.
Searching for songs by The Beatles...

Song 1: "Yesterday"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Here Comes the Sun" by The Beatles...
Done.


## Question 4

4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [11]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import json

def sentiment_analyzed(filename):
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')
    with open(filename, 'r') as f:
        lyrics = json.load(f)
    doc = nlp(lyrics)
    polarity = doc._.blob.polarity
    
    print(f'The Sentiment Polarity of {filename}: {polarity}')
    
sentiment_analyzed('schism_lyrics.json')
sentiment_analyzed('landslide_lyrics.json')
sentiment_analyzed('rain_lyrics.json')
sentiment_analyzed('sun_lyrics.json')

The Sentiment Polarity of schism_lyrics.json: 0.1860615079365079
The Sentiment Polarity of landslide_lyrics.json: 0.02091503267973856
The Sentiment Polarity of rain_lyrics.json: -0.09509803921568627
The Sentiment Polarity of sun_lyrics.json: -0.14791666666666667


It feels strange to measure to sentiment of lyrics without including the music in some way. Schism and Landslide are very different songs melodically but have a similarly slightly postive score. The songs do have a slight element of hope so this makes sense. I do not agree that Box of Rain and Here Comes the Sun are slightly negative.